# Python Worksheet: Maximum Likelihood for a Nonlinear Problem

Dear students,

My team and I tried hard to get a useful exercise for MLE. MLE is a crucial tool for all data science. In addition, our standard problem set was handed out as part of the python lecture notes. We hence had to come up with something useful, yet, also doable at the BSc level.

The following exercise is the result. We will likely spend some time discussing your and my work during the upcoming prof cafe.


 


## Remember this Crucial FACT

**Do NOT numerically optimize a Gaussian likelihood for a LINEAR model. Instead, take the ML parameter estimates from an OLS fit AND the ML estimate for the volatility from the respective residual**

$$
\sqrt{\frac{1}{T} \epsilon_{ols}' \epsilon_{ols}}
$$

##  A Useful Exercise had to be Non-Linear

Reason: see above

## G.0 Set-up of the Python Challenge

Have you ever wondered how central banks, banks, hedge funds and investors know whether a government bond is fairly priced, overpriced or undervalued? Well, they use the approach of "net present value" to figure out what the price of a bond should be. To do so, one has to get accurate forecasts of how the short rate and risk premia move into the future. A popular approach relies on so called "short rate models". These models fit the time series of the short rate to a realistic, yet convenient, parametrization. This allows to obtain a forecast of where short rates will be in the future. You need to do the same for the risk premium. You might wonder why do government bonds pay a risk premium. Well, the longer the maturity of the bond, the higher the interest rate sensitivity and hence, the higher (usually) the (maturity) premium. That premium compensates for the fact that future realized interest rates will be different from todays expected future rates. Based on the CAPM, you could say that interest rate risk is systematic and hence pays a risk premium. That is for the background.

A very popular short rate model is called "Vasicek model". That is because a mathematician, Dr. Vasicek, derived in the 1980s the term structure of interest rates under the assumption that the short rate follows a Gaussian distribution. It is fair to says that the assumed short rate follows a G-LRM. Based on stochastic calculus and the principle of net present value and no arbitrage, Dr. Vasicek derived the price of risk-free (non defaultable) bonds for different maturities. 

The Gaussian PDF of the short rate is

$$
r_t | F_{t-1} \sim \mathcal N\left(\theta^P (1- e^{-\kappa^P})+e^{-\kappa^P} r_{t-1}, \sigma^2 * \frac{1-e^{-2\kappa^P}}{2\kappa^P}\right).
$$

Note, $\theta^P$ is the long-run mean of $r$, $\kappa^P$ is the speed of mean reversion and $\sigma$ is the instantaneous standard deviation of the interest rate shock. As the model of Vasicek is written in continuous time, the above PDF takes several exponentials and ratios, which are the result of an integration that is NOT subject to our BSc course.

Also, Dr. Vasicek derived that the arbitrage-free price of a bond at time $t$ with maturity in $\tau$ periods, $P_t(\tau)$ coincides with

$$
P_t(\tau) = e^{A(\tau) - B(\tau) \times r_t}
$$

with
\begin{align*}
B(\tau) &:= \frac{1-e^{-\kappa^Q \tau}}{\kappa^Q} \\
A(\tau) &:= (\theta^Q - \frac{\sigma^2}{2 (\kappa^Q)^2}) \times (  B(\tau) - \tau ) - \frac{\sigma^2}{4\kappa^Q} B^2(\tau) 
\end{align*}


Note, from the CAPM and NPV you know that prices of financial assets are discounted by the risk-free rate plus risk premium. The parameters $\theta^Q$ and $\kappa^Q$ differ from $\theta^P$ and $\kappa^P$, respectively. The former are adjusted by risk premiums. Intuitively, it is correct to think that the former are risk premium adjusted expressions of the latter. E.g. if the long-run mean of the short rate was at 2\%, it could be that bonds are priced as if the long-run rate was at 3\%. This means the price is lower than if the future payoff was discounted by 2\%. That extra reduction in price can be correctly interpreted as the result of a risk premium. That was to show you that the so called "Q parameters" are risk premium adjusted (and extracted from prices), while the $P$ parameters are extracted from empirical time-series data. I provide a formal and intuitive introduction into these asset pricing concepts in an upcoming MSc class on financial machine learning.

For now, you need only to keep in mind that the parameters of the model are $\kappa^P, \theta^P$ for the short rate PDF and $\theta^Q, \kappa^Q$ for the bond prices. The parameter $\sigma$ affects both, the short rate PDF and the bond price PDF.

Note, prices are non-stationary! Hence, we work with continuousy compounded yields
\begin{align*}
y_t(\tau) &:= - \frac{1}{\tau} \ln P_t(\tau)  
\end{align*}
 




## G.1 Data

You will work with the same interest rate data than a couple of weeks ago. The rates are in "GovBondYields.xls". 

Take the 12-, 60 and 120 months interest rates as your yields of interest, $y$. Take the 3-month yield as the short rate $r$.

**Units:** Divide the rates data by 1200 to arrive at monthly values in decimals. Work with these units.

 


## G.2 Financial Data Science Challenge

Your data science task is to extract the time-series of the expected (interest rate) risk premium that is priced-into government bonds. 
 


## G.3 Battle Plan

The battle plan is as follows. 

$$
\\
$$

1. Write down the joint log likelihood function for $r_t$ and $y_t(\tau), \tau \in [12,60,120]$. For that assume a pairwise independent Gaussian measurement error for all yields. Further assume each of these measurement errors shares the same volatility, which we call
$$
\sigma_y.
$$
Note, $\sigma_y$ will be part of the parameter vector over which you optimize.

Note, you can interpret observed yields as Vasicek (model)-implied yield $y_t(\tau)$ plus a Gaussian measurement error that is drawn from $N(0,\sigma^2_y)$.

Note: the above assumption implies that the joint log likelihood function can be treated as the sum of 4 pairwise independent log likelihood functions. The independence refers to the respective residual being independent of the residuals of the other log likelihood. The parameters do of course affect all of the log likelihoods; although strictly speaking, $\kappa^P, \theta^P$ affect only the log likelihood for $r$, while $\kappa^Q, \theta^Q$ affect only the log likelhood for the 3 yields.


$$
\\
$$

2. Write a python function that implements the joint log likelihood function. More recent research has shown you could indeed fit the PDF for $r$ independently for the PDF of $y$. Yet, here, we optimize the joint log likelihood.

**Hint:** Multivariate Gaussian likelihood: https://en.wikipedia.org/wiki/Multivariate_normal_distribution

yet, yours will look simpler as we made some simplifying assumptions. 


$$
\\
$$

3. Play a bit around with starting values for the optimization. That simple example reveals already, that you need to choose starting values carefully to end up in the global optimum. Regardless of your playing around, use at least the following procedure to find smart starting values

3.1 fit $r$ to an OLS and recover the ML estimates for $\kappa^P, \theta^P, \sigma$. Make these parameters be the respective starting values $\kappa^P, \theta^P, \sigma$ in  the joint log likelihood optimization. 

3.2 for the starting values of the Q parameters, we assume a 0 risk premium. Hence $\kappa^Q_{startValue} = \kappa^P_{startValue}$ and $\theta^Q_{startValue} = \theta^P_{startValue}$. Double check: inside the optimization, Q and P parameters can differ(!). P parameters affect $r$ while $Q$ parameters affect yields (!)

3.3 For the starting value of $\sigma_y$ we do the following: Regress (OLS) the 60month yield onto $r$ and take the volatility of the residual as the starting value for $\sigma_y$, i.e. (np.sqrt( (...).mse_resid)) where we use the MSE_RESID function of stats model's OLS package. 

**Hint (1):** The rounded ols starting values coincide with 0.014895, 0.00466, 0.014895, 0.00466, 0.00037794, 0.000777 for $\kappa^P$, $\theta^P$, $\kappa^Q$, $\theta_Q$, $\sigma$, $\sigma_y$, respectively.

**Hint (2):** The negative of the joint log likelihood function (over $r$ and $y$), evaluated at   the OLS starting values coincides with -1305x.xxxx (around -13k) 


$$
\\
$$

4. The optimization routine is similar to the ipynb from class. Yet, you will learn that the local optimizer fails to find parameters that beat the starting values. 

4.1 use sco.minimize

4.2 use L-BFGS-B as the optimization routine

4.3 use a 'gtol' and 'ftol* of 1e-12

4.4 minimize the negative joint log likelihood function

4.6 the upper bound for all parameters is **1**. 

4.7 the lower bound for all parameters is **-1**, except for $\sigma$ and $\sigma_y$ whose lower bound we set to **0.00001**.


$$
\\
$$

5. That nonlinear problem requires a global optimization routine. Hence, redo the optimization with the following numerical optimizer: 

from scipy.optimize import dual_annealing

results_annealing = dual_annealing(func=likelihood_neg, args=(rate_df, yield_df), bounds=bnds_all, seed=19937, maxiter=2000)

where likelihood_neg is the function that computes the negative join log likelihood, args = (...) are the pandas data frames for the short rate $r$ and the 3 yields $y$ that are used to learn the parameters, and bnds_all is a tuple containing the lower and upper bounds as in 4.6 and 4.7. 


$$
\\
$$


6. Estimate the model parameters of Vasicek using the local optimizer from step 4 and the global one from step 5.

**Hint:** The optimal joint log likelihood function takes the value (for global optimization) of -1481x.xxxx around (-14.8k). 

**Praktomat will ask for:** MLE for $\kappa^P$, $\theta^P$, $\kappa^Q$, $\theta_Q$, $\sigma$, $\sigma_y$, respectively of the local and the global optimization routine.


$$
\\
$$

7. Inside the vasicek model, the priced-in expected risk premium of a $\tau$ maturity bond, $ERP_{t,\tau}$ is

\begin{align*}
ERP_{t,\tau} &= -B(\tau) \times \sigma \times MPR_t \\
MPR_t &:= \lambda_0 + \lambda_1 \times r_t
\end{align*}

with
\begin{align*}
\lambda_0 &:= \frac{\kappa^P \theta^P - \kappa^Q \theta^Q}{\sigma_r} \\
\lambda_1 &:= \frac{\kappa^Q - \kappa^P}{\sigma_r}.
\end{align*}

and $B(\tau)$ being the interest rate loading in the Vasicek derivation of the bond price (i.e. duration of the bond).

Here, you see that spread of Q and P parameters captures risk premium information. A formal explanation of that is not doable in the BSc course on Financial Data Science.

Plot the time series of the expected risk premium for the 1-yr and 10-yr bond. 

**Praktomat will ask for:** $ERP_{0,120}$, $ERP_{10,120}$, $ERP_{100,120}$, $ERP_{500,120}$.


$$
\\
$$

8. Consider the basic return decompostion: $r = \mu + \epsilon$. Plot the respective $\mu$ and $\epsilon$ for the ten-year bond. Compare it to the return decomposition that you have seen for equity returns.

**Hint:**
\begin{align*}
\mu &:= r_t + ERP_{t,\tau}|_{\tau = 120} \\
\epsilon &:= -B(\tau)|_{\tau=120} \times  \epsilon^{r}
\end{align*}

where $\epsilon^{r}$ is the realized innovation (i.e. residual) to the short rate. 

**Hint:** you get $\epsilon^{r}$ from the time series of $r_t$ and its respective Gaussian PDF.

**Hint on Hint:** Take the MLE implied interest rate innovation (residual) and you found the time series of $\epsilon^r$.

Note: Ensure you see why a surprise increase in the short rate $\epsilon^{r}>0$ leads to a negative bond return (i.e. higher discount rate than expected, lower price than expected).


**Praktomat will ask for:** $\mu[0], \mu[1], \mu[100], \mu[500]$ and $\epsilon[0], \epsilon[1], \epsilon[100], \epsilon[500]$.

$$
\\
$$

9. As you plot the ten-year bond yield in data and the vasicek implied ten-year bond yield (from the global optimiztion) you will notice that the one factor model of vasicek has problems fitting longer-term yields. This happens because Vasicek uses only one factor to explain bond yields, while one needs at least two factors (better three) to explain movements in short-, medium- and long-term bond yields. A generalization to three factors is not the scope of this worksheet, yet, it is straight-forward to do so, if you have successfully completed this worksheet.

In [3]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import statsmodels.api as sm


In [4]:
data = pd.read_excel('GovBondYields.xls')

r_df = data.__getitem__(3)
y_1 = data.__getitem__(12)
y_2 = data.__getitem__(60)
y_3 = data.__getitem__(120)

y_data = pd.DataFrame(y_1.values)/1200
y_data['60'] = y_2.values/1200
y_data['120'] = y_3.values/1200

T = len(r_df) -1

helpr = np.matrix(r_df).reshape(T+1,1)/1200
r = helpr[1:T+1]
# print(r)

r_lag = helpr[0:T]

y_values = np.matrix(y_data)
y_60_reg = y_values[:,1]
y_values = y_values[1:]


1. Write down the joint log likelihood function for $r_t$ and $y_t(\tau), \tau \in [12,60,120]$. For that assume a pairwise independent Gaussian measurement error for all yields. Further assume each of these measurement errors shares the same volatility, which we call
$$
\sigma_y.
$$
Note, $\sigma_y$ will be part of the parameter vector over which you optimize.

Note, you can interpret observed yields as Vasicek (model)-implied yield $y_t(\tau)$ plus a Gaussian measurement error that is drawn from $N(0,\sigma^2_y)$.

Note: the above assumption implies that the joint log likelihood function can be treated as the sum of 4 pairwise independent log likelihood functions. The independence refers to the respective residual being independent of the residuals of the other log likelihood. The parameters do of course affect all of the log likelihoods; although strictly speaking, $\kappa^P, \theta^P$ affect only the log likelihood for $r$, while $\kappa^Q, \theta^Q$ affect only the log likelhood for the 3 yields.

$$
\\
$$

The Gaussian PDF of the short rate is

$$
r_t | F_{t-1} \sim \mathcal N\left(\theta^P (1- e^{-\kappa^P})+e^{-\kappa^P} r_{t-1}, \sigma^2 * \frac{1-e^{-2\kappa^P}}{2\kappa^P}\right).
$$

Note, $\theta^P$ is the long-run mean of $r$, $\kappa^P$ is the speed of mean reversion and $\sigma$ is the instantaneous standard deviation of the interest rate shock. As the model of Vasicek is written in continuous time, the above PDF takes several exponentials and ratios, which are the result of an integration that is NOT subject to our BSc course.

Also, Dr. Vasicek derived that the arbitrage-free price of a bond at time $t$ with maturity in $\tau$ periods, $P_t(\tau)$ coincides with

$$
P_t(\tau) = e^{A(\tau) - B(\tau) \times r_t}
$$

with
\begin{align*}
B(\tau) &:= \frac{1-e^{-\kappa^Q \tau}}{\kappa^Q} \\
A(\tau) &:= (\theta^Q - \frac{\sigma^2}{2 (\kappa^Q)^2}) \times (  B(\tau) - \tau ) - \frac{\sigma^2}{4\kappa^Q} B^2(\tau) 
\end{align*}




For now, you need only to keep in mind that the parameters of the model are $\kappa^P, \theta^P$ for the short rate PDF and $\theta^Q, \kappa^Q$ for the bond prices. The parameter $\sigma$ affects both, the short rate PDF and the bond price PDF.

Note, prices are non-stationary! Hence, we work with continuousy compounded yields
\begin{align*}
y_t(\tau) &:= - \frac{1}{\tau} \ln P_t(\tau)  
\end{align*}


In [5]:
def neg_log_likelihood(x):
    """
    Input: x, a vector containg theta^P, kappa^P,sigma^2, theta^Q, kappa^Q and sigma_y^2
    """
    #Read parameters from x
    theta_p = x[0]
    kappa_p = x[1]
    sigma = x[2]**2#This is actually sigma^2
    theta_q = x[3]
    kappa_q = x[4]
    sigma_y = x[5]**2#This is actually sigma_y^2
    #Calculate ln L of r
    r_var = sigma * ((1 - np.exp(-2 * kappa_p)) / (2 * kappa_p))

    r_help = r - (theta_p * (1- np.exp(-kappa_p)) * np.ones(T).reshape(T,1) + np.exp(-kappa_p) * r_lag)
    #print('mu= ', r_help)
    #print('sigma r = ', r_var)
    # print(r_help.shape)
    ln_L_r = -(T/2) * np.log(2 * np.pi * r_var) - (r_help.T @ r_help ) / (2*r_var)
    # print(ln_L_r)

    #Calculate ln L of y(tau)
    taus = [12,60,120]
    ln_L_y = np.empty(3)

    for i in range(3):
        tau = taus[i]
        y_obs = y_values[:,i]
        B = (1- np.exp(-kappa_q * tau)) / kappa_q
        A = (theta_q - sigma / (2*(kappa_q**2))) * (B - tau) - (sigma / (4*kappa_q))*(B**2)
        #P = np.exp(A *np.ones(T).reshape(T,1) - B*r)

        #y_t = - (1/tau) * np.log(P) 
        y_t = -(1/tau) * A*np.ones(T).reshape(T,1) + (1/tau)*B*r
        # print(y_t.shape)
        print(y_t.mean())
        
        ln_L_y_tau = -(T/2) * np.log(2*np.pi * sigma_y) - ((y_obs - y_t).T @ (y_obs - y_t)) / (2*sigma_y)
        print(ln_L_y_tau)
        
        ln_L_y[i] = ln_L_y_tau

    #print('ln L(r), ', ln_L_r)
    #print('ln L(y(12)): ', ln_L_y[0])
    #print('ln L(y(60)): ', ln_L_y[1])
    #print('ln L(y(120)): ', ln_L_y[2])

    return -(ln_L_r + np.sum(ln_L_y))[0,0]

3. Play a bit around with starting values for the optimization. That simple example reveals already, that you need to choose starting values carefully to end up in the global optimum. Regardless of your playing around, use at least the following procedure to find smart starting values

3.1 fit $r$ to an OLS and recover the ML estimates for $\kappa^P, \theta^P, \sigma$. Make these parameters be the respective starting values $\kappa^P, \theta^P, \sigma$ in  the joint log likelihood optimization. 

3.2 for the starting values of the Q parameters, we assume a 0 risk premium. Hence $\kappa^Q_{startValue} = \kappa^P_{startValue}$ and $\theta^Q_{startValue} = \theta^P_{startValue}$. Double check: inside the optimization, Q and P parameters can differ(!). P parameters affect $r$ while $Q$ parameters affect yields (!)

3.3 For the starting value of $\sigma_y$ we do the following: Regress (OLS) the 60month yield onto $r$ and take the volatility of the residual as the starting value for $\sigma_y$, i.e. (np.sqrt( (...).mse_resid)) where we use the MSE_RESID function of stats model's OLS package. 

**Hint (1):** The rounded ols starting values coincide with 0.014895, 0.00466, 0.014895, 0.00466, 0.00037794, 0.000777 for $\kappa^P$, $\theta^P$, $\kappa^Q$, $\theta_Q$, $\sigma$, $\sigma_y$, respectively.

**Hint (2):** The negative of the joint log likelihood function (over $r$ and $y$), evaluated at   the OLS starting values coincides with -1305x.xxxx (around -13k) 

In [8]:
result = sm.OLS(r,sm.add_constant(r_lag)).fit()
beta1, beta2 = result.params
sigma_ols = (result.mse_resid)
kappa_P = -np.log(beta2)
theta_P = (beta1 / (1 - beta2))
Sigma = np.sqrt(sigma_ols * 2 * kappa_P / (1- np.exp(-2*kappa_P)))

result2 = sm.OLS(y_values[:,1],sm.add_constant(r)).fit()
Sigma_y = np.sqrt(result2.mse_resid)

theta_P = 0.00466
kappa_P = 0.014895
Sigma = 0.00037794
Sigma_y = 0.000777

print('Theta^P = ', theta_P)
print('Kappa^P = ',kappa_P)
print('Sigma = ',Sigma)
print('Sigma_y = ',Sigma_y)

print('')
print('Negative joint log likelihood = ', neg_log_likelihood([theta_P, kappa_P, 0.00037794, theta_P, kappa_P, Sigma_y]))

Theta^P =  0.00466
Kappa^P =  0.014895
Sigma =  0.00037794
Sigma_y =  0.000777

(624, 1)
0.0043616007362109925
[[3682.7932414]]
(624, 1)
0.004400522674479373
[[2950.03645286]]
(624, 1)
0.004400203122405187
[[2373.7643727]]
Negative joint log likelihood =  -13039.759559177639


4. The optimization routine is similar to the ipynb from class. Yet, you will learn that the local optimizer fails to find parameters that beat the starting values. 

4.1 use sco.minimize

4.2 use L-BFGS-B as the optimization routine

4.3 use a 'gtol' and 'ftol* of 1e-12

4.4 minimize the negative joint log likelihood function

4.6 the upper bound for all parameters is **1**. 

4.7 the lower bound for all parameters is **-1**, except for $\sigma$ and $\sigma_y$ whose lower bound we set to **0.00001**.


In [9]:
bound = Bounds(np.array([-1,-1,1e-5,-1,-1,1e-5]),np.array([1,1,1,1,1,1]))
x_0 = np.array([theta_P,kappa_P,Sigma,theta_P,kappa_P,Sigma_y])
result = minimize(fun=neg_log_likelihood,x0=x_0,method='L-BFGS-B',bounds=bound, options={'ftol' : 1e-12, 'gtol' : 1e-12})
print(result)
theta_p_local = result.x[0]
kappa_p_local = result.x[1]
sigma_local =result.x[2]
theta_q_local =result.x[3]
kappa_q_local =result.x[4]
sigma_y_local =result.x[5]
print('theta_p_local = ', theta_p_local)
print('kappa_p_local = ', kappa_p_local)
print('sigma_local = ', sigma_local)
print('theta_q_local = ', theta_q_local)
print('kappa_q_local = ', kappa_q_local)
print('sigma_y_local = ', sigma_y_local)

(624, 1)
0.0043616007362109925
[[3682.7932414]]
(624, 1)
0.004400522674479373
[[2950.03645286]]
(624, 1)
0.004400203122405187
[[2373.7643727]]
(624, 1)
0.0043616007362109925
[[3682.7932414]]
(624, 1)
0.004400522674479373
[[2950.03645286]]
(624, 1)
0.004400203122405187
[[2373.7643727]]
(624, 1)
0.0043616007362109925
[[3682.7932414]]
(624, 1)
0.004400522674479373
[[2950.03645286]]
(624, 1)
0.004400203122405187
[[2373.7643727]]
(624, 1)
0.004361600577211221
[[3682.79316642]]
(624, 1)
0.004400520229260898
[[2950.03410597]]
(624, 1)
0.0044001973260183605
[[2373.75780267]]
(624, 1)
0.004361601578961087
[[3682.79363882]]
(624, 1)
0.004400526063078835
[[2950.03970517]]
(624, 1)
0.004400208464213574
[[2373.77042745]]
(624, 1)
0.004361600753668627
[[3682.7932099]]
(624, 1)
0.004400522746668927
[[2950.03627518]]
(624, 1)
0.004400203257823829
[[2373.76407567]]
(624, 1)
0.0043616007362109925
[[3682.79065893]]
(624, 1)
0.004400522674479373
[[2950.05273119]]
(624, 1)
0.004400203122405187
[[2373.79548

5. That nonlinear problem requires a global optimization routine. Hence, redo the optimization with the following numerical optimizer: 

from scipy.optimize import dual_annealing

results_annealing = dual_annealing(func=likelihood_neg, args=(rate_df, yield_df), bounds=bnds_all, seed=19937, maxiter=2000)

where likelihood_neg is the function that computes the negative join log likelihood, args = (...) are the pandas data frames for the short rate $r$ and the 3 yields $y$ that are used to learn the parameters, and bnds_all is a tuple containing the lower and upper bounds as in 4.6 and 4.7. 

In [ ]:
def neg_log_likelihood_dual(x, rate_df, yields_df):
    helpr = np.matrix(rate_df).reshape(T+1,1)/1200
    r = helpr[1:T+1]

    r_lag = helpr[0:T]

    y_values = np.matrix(y_data)
    y_values = y_values[1:]

    return neg_log_likelihood(x)

In [ ]:
print(neg_log_likelihood_dual(x_0, r_df,y_data))

In [ ]:
bnds_all = [(-1,1),(-1,1),(1e-5,1),(-1,1),(-1,1),(1e-5,1)]
results_annealing = dual_annealing(func=neg_log_likelihood_dual, args=(r_df, y_data), bounds=bnds_all, seed=19937, maxiter=2000)
print(results_annealing)

theta_p_global = results_annealing.x[0]
kappa_p_global = results_annealing.x[1]
sigma_global =results_annealing.x[2]
theta_q_global =results_annealing.x[3]
kappa_q_global =results_annealing.x[4]
sigma_y_global =results_annealing.x[5]
print('theta_p_global = ', theta_p_global)
print('kappa_p_global = ', kappa_p_global)
print('sigma_global = ', sigma_global)
print('theta_q_local = ', theta_q_local)
print('kappa_q_global = ', kappa_q_global)
print('sigma_y_global = ', sigma_y_global)


7. Inside the vasicek model, the priced-in expected risk premium of a $\tau$ maturity bond, $ERP_{t,\tau}$ is

\begin{align*}
ERP_{t,\tau} &= -B(\tau) \times \sigma \times MPR_t \\
MPR_t &:= \lambda_0 + \lambda_1 \times r_t
\end{align*}

with
\begin{align*}
\lambda_0 &:= \frac{\kappa^P \theta^P - \kappa^Q \theta^Q}{\sigma_r} \\
\lambda_1 &:= \frac{\kappa^Q - \kappa^P}{\sigma_r}.
\end{align*}

and $B(\tau)$ being the interest rate loading in the Vasicek derivation of the bond price (i.e. duration of the bond).

Here, you see that spread of Q and P parameters captures risk premium information. A formal explanation of that is not doable in the BSc course on Financial Data Science.

Plot the time series of the expected risk premium for the 1-yr and 10-yr bond. 

**Praktomat will ask for:** $ERP_{0,120}$, $ERP_{10,120}$, $ERP_{100,120}$, $ERP_{500,120}$.


In [ ]:
def erp(x,rate_df,yields_df):
    helpr = np.matrix(rate_df).reshape(T+1,1)/1200
    r = helpr[1:T+1]

    r_lag = helpr[0:T]

    y_values = np.matrix(y_data)
    y_values = y_values[1:]

    taus = [12,60,120]
    ln_L_y = np.empty(3)

    theta_p = x[0]
    kappa_p = x[1]
    sigma = x[2]**2#This is actually sigma^2
    theta_q = x[3]
    kappa_q = x[4]
    sigma_y = x[5]**2#This is actually sigma_y^2

    lambda_0 = (kappa_p*theta_p - kappa_q*theta_q)/np.sqrt(sigma)
    lambda_1 = (kappa_q - kappa_p) / np.sqrt(sigma)

    mpr = lambda_0 * np.ones(T).reshape(T,1) + lambda_1 * r

    taus = [12,60,120]
    erp_matrix = np.empty(T*3).reshape(T,3)

    for i in range(3):
        tau = taus[i]
        y_obs = y_values[:,i]
        B = (1- np.exp(-kappa_q * tau)) / kappa_q
        erp_tau = (-B * np.sqrt(sigma)* mpr).reshape(T)
        erp_matrix[:,i] = erp_tau

    return erp_matrix

In [ ]:
erp_res = erp(results_annealing.x,r_df,y_data)
x_ax_val = data.__getitem__('Date')[1:]

plt.plot(x_ax_val,erp_res[:,0], label='12 months')
plt.plot(x_ax_val,erp_res[:,1], label='60 months')
plt.plot(x_ax_val,erp_res[:,2], label='120 months')
plt.legend()
plt.show()

In [ ]:
erp_0_120 = erp_res[0,2]
erp_10_120 = erp_res[10,2]
erp_100_120 = erp_res[100,2]
erp_500_120 = erp_res[500,2]

print('erp_0_120 = ',erp_0_120)
print('erp_10_120 = ',erp_10_120)
print('erp_100_120 = ',erp_100_120)
print('erp_500_120 = ',erp_500_120)

8. Consider the basic return decompostion: $r = \mu + \epsilon$. Plot the respective $\mu$ and $\epsilon$ for the ten-year bond. Compare it to the return decomposition that you have seen for equity returns.

**Hint:**
\begin{align*}
\mu &:= r_t + ERP_{t,\tau}|_{\tau = 120} \\
\epsilon &:= -B(\tau)|_{\tau=120} \times  \epsilon^{r}
\end{align*}

where $\epsilon^{r}$ is the realized innovation (i.e. residual) to the short rate. 

**Hint:** you get $\epsilon^{r}$ from the time series of $r_t$ and its respective Gaussian PDF.

**Hint on Hint:** Take the MLE implied interest rate innovation (residual) and you found the time series of $\epsilon^r$.

Note: Ensure you see why a surprise increase in the short rate $\epsilon^{r}>0$ leads to a negative bond return (i.e. higher discount rate than expected, lower price than expected).


**Praktomat will ask for:** $\mu[0], \mu[1], \mu[100], \mu[500]$ and $\epsilon[0], \epsilon[1], \epsilon[100], \epsilon[500]$.


The Gaussian PDF of the short rate is

$$
r_t | F_{t-1} \sim \mathcal N\left(\theta^P (1- e^{-\kappa^P})+e^{-\kappa^P} r_{t-1}, \sigma^2 * \frac{1-e^{-2\kappa^P}}{2\kappa^P}\right).
$$

In [ ]:
def r_model(r_lag,theta_p,kappa_p):
    return theta_p * (1- np.exp(-kappa_p)) * np.ones(T).reshape(T,1) + np.exp(-kappa_p) * r_lag

mu = r + erp_res[:,2].reshape(T,1)
r_vasicek = r_model(r_lag,theta_p_global,kappa_p_global)
eps_r = r - r_vasicek
tau = 120
B = (1- np.exp(-kappa_q_global * tau)) / kappa_q_global
epsilon = -B * eps_r

for i in [0,1,100,500]:
    print('mu',i,' = ', mu[i,0])

print()

for i in [0,1,100,500]:
    print('epsilon',i,' = ', epsilon[i,0])